# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv('../WeatherPy/Output/CityData.csv')
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,naze,28.3667,129.4833,294.66,81,99,4.18,JP,1622238352
1,sahuaripa,29.0500,-109.2333,312.67,5,2,5.97,MX,1622238495
2,cape town,-33.9258,18.4232,288.64,93,75,0.45,ZA,1622238123
3,yellowknife,62.4560,-114.3525,288.33,41,40,1.34,CA,1622237876
4,ushuaia,-54.8000,-68.3000,272.96,100,90,13.38,AR,1622238496
...,...,...,...,...,...,...,...,...,...
495,kalulushi,-12.8378,28.0919,278.02,56,0,1.63,ZM,1622238637
496,namibe,-15.1961,12.1522,296.49,64,0,1.88,AO,1622238637
497,jackson,42.4165,-122.8345,299.47,25,1,2.06,US,1622238609
498,arcachon,44.5875,-1.0519,294.33,63,0,3.09,FR,1622238638


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=gkey)

In [4]:
coordinates=[]
i = 0
for i in range(city_data['City'].count()):
    thistuple = (city_data['Lat'][i],city_data['Lng'][i])
    coordinates.append(thistuple)

In [5]:
coordinates=city_data[["Lat","Lng"]]
humidity =city_data["Humidity"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
city_data.dropna()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,naze,28.3667,129.4833,294.66,81,99,4.18,JP,1622238352
1,sahuaripa,29.0500,-109.2333,312.67,5,2,5.97,MX,1622238495
2,cape town,-33.9258,18.4232,288.64,93,75,0.45,ZA,1622238123
3,yellowknife,62.4560,-114.3525,288.33,41,40,1.34,CA,1622237876
4,ushuaia,-54.8000,-68.3000,272.96,100,90,13.38,AR,1622238496
...,...,...,...,...,...,...,...,...,...
495,kalulushi,-12.8378,28.0919,278.02,56,0,1.63,ZM,1622238637
496,namibe,-15.1961,12.1522,296.49,64,0,1.88,AO,1622238637
497,jackson,42.4165,-122.8345,299.47,25,1,2.06,US,1622238609
498,arcachon,44.5875,-1.0519,294.33,63,0,3.09,FR,1622238638


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = city_data.dropna()

In [9]:
i = hotel_df['City'].count()
j = 0
hotelName = []
hotelLat = []
hotelLng = []
coordinates2=[]
target_type = "hotel"
radius = 5000
for j in range(hotel_df['City'].count()):
    print(j)
    location = str(hotel_df['Lat'][j])+","+str(hotel_df['Lng'][j])
    #print(location)
    params = { 
        "location": location,
        "types": target_type,
        "radius": radius,
        "key": gkey
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params)
    seattle_bikes = response.json()
    #print(json.dumps(seattle_bikes, indent=4, sort_keys=True))
    hotelName.append(seattle_bikes["results"][1]["name"])
    hotelLat.append(seattle_bikes["results"][1]['geometry']['location']['lat'])
    hotelLng.append(seattle_bikes["results"][1]['geometry']['location']['lat'])
    mytuple = (seattle_bikes["results"][1]['geometry']['location']['lat'],seattle_bikes["results"][1]['geometry']['location']['lat'])
    coordinates2.append(mytuple)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


KeyError: 31

In [ ]:
hotel_df['Hotel Name']=hotelName
hotel_df

In [ ]:
markers = gmaps.marker_layer(coordinates2)
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

         Lat       Lng
0    28.3667  129.4833
1    29.0500 -109.2333
2   -33.9258   18.4232
3    62.4560 -114.3525
4   -54.8000  -68.3000
..       ...       ...
495 -12.8378   28.0919
496 -15.1961   12.1522
497  42.4165 -122.8345
498  44.5875   -1.0519
499  -8.3450  113.6058

[500 rows x 2 columns]


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to updte with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
target_type="hotel"


SyntaxError: invalid syntax (<ipython-input-10-b8681b42412c>, line 1)

In [ ]:
# Add marker layer ontop of heat map


# Display figure
